# Part 2.4 | Reshaping Panel Data

Figure generation notebook

In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.gridspec as gridspec

plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'font.size': 14,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'font.style': 'italic',
    'figure.dpi': 400
})

category_color_pal = sns.color_palette('hls', 8)

In [ ]:
# Load data
percap = pd.read_csv('data/Coffee_Per_Cap.csv', index_col=0)

# Select a subset of countries and years for cleaner visuals
countries = ['USA', 'DEU', 'JPN', 'GBR', 'FRA']
years = ['1999', '2004', '2009', '2014', '2019']
subset = percap[percap['Code'].isin(countries)][['Code'] + years].reset_index(drop=True)
subset

## Figure w_01: Wide Format Table

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
ax.axis('off')

table_data = [['Code'] + years]
for _, row in subset.iterrows():
    table_data.append([row['Code']] + [f"{row[y]:.1f}" for y in years])

table = ax.table(cellText=table_data, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.8)

for j in range(len(years) + 1):
    table[(0, j)].set_facecolor('#4472C4')
    table[(0, j)].set_text_props(color='white', weight='bold')

plt.title('Wide Format: Coffee Consumption (kg per capita)', fontsize=14, pad=20)
plt.tight_layout()
plt.savefig('i/w_01.png', bbox_inches='tight', facecolor='white')

## Figure w_02: Long Format Table

In [ ]:
# Melt to long format
long_df = subset.melt(id_vars=['Code'], var_name='Year', value_name='Consumption')
long_sample = long_df.head(12)

fig, ax = plt.subplots(figsize=(6, 5))
ax.axis('off')

table_data = [['Code', 'Year', 'Consumption']]
for _, row in long_sample.iterrows():
    table_data.append([row['Code'], row['Year'], f"{row['Consumption']:.1f}"])

table = ax.table(cellText=table_data, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.6)

for j in range(3):
    table[(0, j)].set_facecolor('#4472C4')
    table[(0, j)].set_text_props(color='white', weight='bold')

plt.title('Long Format: Coffee Consumption', fontsize=14, pad=20)
plt.tight_layout()
plt.savefig('i/w_02.png', bbox_inches='tight', facecolor='white')

## Figure w_03: Melt Operation Diagram

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4), gridspec_kw={'width_ratios': [3, 1, 2.5]})

# Left: Wide format
ax = axes[0]
ax.axis('off')
small_wide = subset.head(3)[['Code', '1999', '2019']]
table_data = [['Code', '1999', '2019']]
for _, row in small_wide.iterrows():
    table_data.append([row['Code'], f"{row['1999']:.1f}", f"{row['2019']:.1f}"])

table = ax.table(cellText=table_data, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 1.8)
for j in range(3):
    table[(0, j)].set_facecolor('#4472C4')
    table[(0, j)].set_text_props(color='white', weight='bold')
ax.set_title('Wide Format', fontsize=12, pad=10)

# Middle: Arrow
ax = axes[1]
ax.axis('off')
ax.annotate('', xy=(0.9, 0.5), xytext=(0.1, 0.5),
            arrowprops=dict(arrowstyle='->', color='#4472C4', lw=3))
ax.text(0.5, 0.65, 'melt()', ha='center', va='bottom', fontsize=14, style='italic', color='#4472C4')

# Right: Long format
ax = axes[2]
ax.axis('off')
small_long = small_wide.melt(id_vars=['Code'], var_name='Year', value_name='Consumption')
table_data = [['Code', 'Year', 'Consumption']]
for _, row in small_long.iterrows():
    table_data.append([row['Code'], row['Year'], f"{row['Consumption']:.1f}"])

table = ax.table(cellText=table_data, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.1, 1.5)
for j in range(3):
    table[(0, j)].set_facecolor('#4472C4')
    table[(0, j)].set_text_props(color='white', weight='bold')
ax.set_title('Long Format', fontsize=12, pad=10)

plt.suptitle('Melting: Wide → Long', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('i/w_03.png', bbox_inches='tight', facecolor='white')

## Figure w_04: Line Plot from Long Format

In [ ]:
# Full melt for line plot
full_long = percap[percap['Code'].isin(['USA', 'DEU', 'JPN'])].melt(
    id_vars=['Code'], var_name='Year', value_name='Consumption'
)
full_long['Year'] = full_long['Year'].astype(int)

fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data=full_long, x='Year', y='Consumption', hue='Code',
             palette=category_color_pal[:3], linewidth=2, ax=ax)

plt.title('Coffee Consumption Over Time (from Long Format)', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Consumption (kg per capita)')
plt.legend(title='Country')
sns.despine()
plt.tight_layout()
plt.savefig('i/w_04.png', bbox_inches='tight')

## Figure w_05: Pivot Operation Diagram

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4), gridspec_kw={'width_ratios': [2.5, 1, 3]})

# Left: Long format
ax = axes[0]
ax.axis('off')
small_wide = subset.head(3)[['Code', '1999', '2019']]
small_long = small_wide.melt(id_vars=['Code'], var_name='Year', value_name='Consumption')
table_data = [['Code', 'Year', 'Consumption']]
for _, row in small_long.iterrows():
    table_data.append([row['Code'], row['Year'], f"{row['Consumption']:.1f}"])

table = ax.table(cellText=table_data, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.1, 1.5)
for j in range(3):
    table[(0, j)].set_facecolor('#4472C4')
    table[(0, j)].set_text_props(color='white', weight='bold')
ax.set_title('Long Format', fontsize=12, pad=10)

# Middle: Arrow
ax = axes[1]
ax.axis('off')
ax.annotate('', xy=(0.9, 0.5), xytext=(0.1, 0.5),
            arrowprops=dict(arrowstyle='->', color='#4472C4', lw=3))
ax.text(0.5, 0.65, 'pivot()', ha='center', va='bottom', fontsize=14, style='italic', color='#4472C4')

# Right: Wide format
ax = axes[2]
ax.axis('off')
table_data = [['Code', '1999', '2019']]
for _, row in small_wide.iterrows():
    table_data.append([row['Code'], f"{row['1999']:.1f}", f"{row['2019']:.1f}"])

table = ax.table(cellText=table_data, loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 1.8)
for j in range(3):
    table[(0, j)].set_facecolor('#4472C4')
    table[(0, j)].set_text_props(color='white', weight='bold')
ax.set_title('Wide Format', fontsize=12, pad=10)

plt.suptitle('Pivoting: Long → Wide', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('i/w_05.png', bbox_inches='tight', facecolor='white')

## Figure w_06: Scatterplot from Wide Format

In [ ]:
fig = plt.figure(figsize=(10, 5))
gs = gridspec.GridSpec(1, 3, figure=fig, width_ratios=[1, 2.4, 1])
ax = fig.add_subplot(gs[0, 1])

sns.scatterplot(data=percap, x='1999', y='2019', color='grey', edgecolors='#4472C4', alpha=0.5, s=100, ax=ax)
ax.plot([0, 20], [0, 20], linestyle='--', color='red', alpha=0.5, zorder=-1)

ax.set_xlabel('Consumption in 1999 (kg per capita)')
ax.set_ylabel('Consumption in 2019 (kg per capita)')
ax.set_title('Comparing Years (from Wide Format)', fontsize=14)
ax.set_xlim(0, 20)
ax.set_ylim(0, 20)
ax.grid(True, color='lightgray', linestyle='--', linewidth=0.4)

plt.tight_layout()
plt.savefig('i/w_06.png', bbox_inches='tight')